$ min \, f(\vec{x} ) = -20 \cdot e^{\left(-0.2\sqrt{\frac{1}{n} \sum^{n}_{i=1} x^2_1} \right)} -e^{\left( \frac{1}{n} \sum^n_{i=1} cos(2\pi x_i \right)} + 20 + e$     
donde $|x_i| <= 30$

In [2]:
from math import e,cos,pi
import numpy as np
from numpy.random import normal, uniform
from random import sample
from statistics import mean, pstdev
from numpy import ndarray as nd

In [3]:
def f(x):
    n = len(x)
    #try:
    return -20*e**(-0.2*((1/n)*sum(x_i**2 for x_i in x))) - e**((1/n)*sum(cos(2*pi*x_i) for x_i in x)) + 20 + e     
    #except:
    #    print(x)

In [3]:
f([8.63110648, -29.11404831])

21.680282958904787

## (1+1)-EE (20 puntos)

In [4]:
def random_solution(n):
    parent = []
    x_s = []
    for i in range(n):
        x_s.append(uniform(-30,30))
    sigma = uniform()
    return [np.array(x_s), sigma, f(x_s)]

In [33]:
random_solution(2)

[array([-27.92920613, -18.56055323]), 0.029994013889005577, 21.731098608822855]

In [5]:
def check_domain(x,lower_limit, upper_limit):
    x = list(x)
    for i in range(len(x)):
        if x[i] < lower_limit:
            x[i] = lower_limit
        elif x[i] > upper_limit:
            x[i] = upper_limit
    return np.array(x)

In [6]:
def mutate_sol(solution):
    x, sigma, fx = solution
    x = list(x)
    for i in range(len(x)):
        x[i] += sigma*normal(0,1) 
    x = check_domain(x,-30,30)
    return [x,sigma,f(x)]

In [75]:
def EE1plus1(n, sigma, c, gens, k):
    best_sol = random_solution(n)
    success_count = 0
    for i in range(gens):
        mutated_sol = mutate_sol(best_sol)
        if mutated_sol[2] < best_sol[2]:
            best_sol = mutated_sol
            success_count += 1
        if i%k == 0:
            success_rate = success_count/k
            if success_rate > 1/5:
                sigma /= c
            elif success_rate < 1/5:
                sigma *= c
            success_count = 0
    return best_sol

In [8]:
EE1plus1(2,uniform(),.8,1000,50)

[array([  6.00544079, -27.01208139]), 0.8676677852427007, 20.00470401626995]

In [9]:
print('Best f =',min([EE1plus1(2,uniform(),.8,1000,50)[2] for i in range(100)]))

Best f = 0.0007485967069951194


### Statistical analysis

In [99]:
def m_EE1plus1(m, n, sigma, c, gens, k):
    m_solutions = []
    for i in range(m):
        EE_sol = EE1plus1(n, sigma, c, gens, k)
        m_solutions.append(EE_sol)
    selection(m, m_solutions)
    return m_solutions

In the function above I've already ordered the solutions from best to worst.

In [137]:
m_EE1plus1(5, 2,uniform(),.8,200,50)

[[array([  2.99560315, -26.00442981]), 0.11202321905933876, 20.00104484513727],
 [array([ 26.0021537 , -15.99167819]), 0.1257882677182638, 20.001981223714516],
 [array([16.0050521 , 17.98818805]), 0.30047358074109753, 20.0044225320462],
 [array([-7.02004339, 14.99545425]), 0.5160613639174717, 20.01129453199101],
 [array([-28.01828412,   1.97363859]), 0.5933677352981471, 20.027420954209813]]

In [138]:
_20sols = m_EE1plus1(20, 2,uniform(),.8,200,50)

In [102]:
def median_sol(m,solutions,one_solution=True):
    if m%2 == 0:
        if one_solution:
            return solutions[m//2]
        return solutions[(m//2)-1] + solutions[(m//2)]
    else:
        return solutions[m//2]

In [139]:
median_sol(20,_20sols)

[array([ -6.00030162, -25.01242144]), 0.3274224894360843, 20.00413659611303]

In [104]:
def mean_sol_f(solutions):
    return mean([s[2] for s in solutions])

In [140]:
mean_sol_f(_20sols)

19.103415393579645

In [106]:
def std_sol_f(solutions):
    return pstdev([s[2] for s in solutions])

In [141]:
std_sol_f(_20sols)

3.9719034812616822

In [109]:
def m_EE1plus1_statistical_analysis(m, n, sigma, c, gens, k):
    m_sols =  m_EE1plus1(m, n, sigma, c, gens, k)
    print('Best solution:',m_sols[0])
    print('Worst solution:', m_sols[-1])
    print('Median solution:', median_sol(m,m_sols))
    print('Mean solution:', mean_sol_f(m_sols))
    print('Standard deviation:', std_sol_f(m_sols))

In [142]:
m = 20
m_EE1plus1_statistical_analysis(m, 2, uniform(), 0.8, 100, 50)

Best solution: [array([-17.99915386,  -5.99912772]), 0.015101250871194294, 20.000039620562656]
Worst solution: [array([-25.08480606, -11.97261121]), 0.8846091395913175, 20.20070855675633]
Median solution: [array([-23.99470234, -13.02348351]), 0.15472882240245622, 20.01547700321343]
Mean solution: 20.032690975919916
Standard deviation: 0.0463953229845462


## $(\mu, \lambda)- EE$

In [7]:
def rand_select_parents(n, population, n_parents):
    parents_index = sample([i for i in range(n)], n_parents)
    return [population[parents_index[0]], population[parents_index[1]]]

In [11]:
rand_select_parents(9,[21,41,44,4323,37,45,3,44,43],2)

[45, 21]

In [8]:
def discrete_offspring(n, parents):
    x_s = []
    sigma = 0
    for i in range(n):
        if uniform() < 0.5:
            x_s.append(parents[0][0][i])
        else:
            x_s.append(parents[1][0][i])
    if uniform() < 0.5:
        sigma = parents[0][1]
    else:
        sigma = parents[1][1]
    return [np.array(x_s), sigma, 0]#not necessary to calculate f(x) because we still have to mutate after this, we can  calculate it then.    

In [130]:
discrete_offspring(3,[ [ [3,4,5],4,4 ], [ [6,7,8],3,3 ] ])

[array([3, 7, 5]), 4, 0]

Mutation with a unique step size:    
$\sigma' = \sigma \cdot e^{\tau \, \cdot N(0,1)}$      
$x'_i = x_i + \sigma'_i \cdot N(0,1)$      
$ \tau = \frac{1}{\sqrt{n}}$

In [26]:
def mutate(offspring_i, sigma, n, tau=None):
    if tau == None: tau = 1/(n**(1/2)) 
    sigma *= (e**(tau*normal())) 
    x_s = []
    for i in range(n):
        new_x_i = offspring_i[0][i] + sigma * normal()
        x_s.append(new_x_i)
    return [np.array(x_s), sigma, f(x_s)] 

In [27]:
rand_sol = random_solution(2)
mutated_solution = mutate(rand_sol, uniform(), 2)
print(rand_sol)
print(mutated_solution)

[array([4.145435  , 6.12755319]), 0.7607054530962822, 20.712624827773865]
[array([3.808272  , 6.40919204]), 0.4778660942870871, 21.85593980951216]


In [28]:
def selection(new_size, population):
    population.sort(key=lambda x: x[2])
    return population[:new_size]

In [29]:
def EEmu_coma_lambda(n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None):
    population = [random_solution(n) for i in range(mu)]
    for gen in range(gens):
        offspring = []
        for kid in range(lambda_):
            parents = rand_select_parents(mu, population, 2) # here can I select the same parent twice? I guess because there are more children than parents.
            offspring_i = discrete_offspring(n,parents)
            offspring_i = mutate(offspring_i,sigma,n,tau)
            offspring_i[0] = check_domain(offspring_i[0],-30,30)
            offspring.append(offspring_i)
        population = selection(lambda_, offspring)
    return population[0]

In [31]:
EEmu_coma_lambda(2, 200, 0.0001, 100, 700)

[array([ 0.00244562, -0.00055912]),
 0.020243898010549627,
 0.00018142732297876663]

### Statistical analysis

In [118]:
def m_EEmu_coma_lambda(m, n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None):
    m_solutions = []
    for i in range(m):
        EE_sol = EEmu_coma_lambda(n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None)
        m_solutions.append(EE_sol)
    selection(m, m_solutions)
    return m_solutions

In the function above I've already ordered the solutions from best to worst.

In [119]:
m_EEmu_coma_lambda(5, 2, 200, 0.0001, 100, 700)

[[array([0.00168585, 0.00181241]), 0.0976762645893737, 0.0001766221274164259],
 [array([-0.0001392 ,  0.00682427]), 0.2187347516689292, 0.001342637463477292],
 [array([-0.01560875, -0.01571606]),
  0.33621981498399894,
  0.014101581925470175],
 [array([0.01194344, 0.02343068]), 0.23252726045280922, 0.019847498157624432],
 [array([-0.0224055 ,  0.01970169]),
  0.11047812768240449,
  0.025522052643598858]]

In [122]:
def m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None):
    m_sols =  m_EEmu_coma_lambda(m, n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None)
    print('Best solution:',m_sols[0])
    print('Worst solution:', m_sols[-1])
    print('Median solution:', median_sol(m,m_sols))
    print('Mean solution:', mean_sol_f(m_sols))
    print('Standard deviation:', std_sol_f(m_sols))

In [123]:
m = 15
m_EEmu_coma_lambda_statistical_analysis(m, 2, 20, 0.0001, 100, 700)

Best solution: [array([ 0.00104194, -0.00283588]), 0.09718292626749558, 0.00026312406108930375]
Worst solution: [array([-0.00555778, -0.05898878]), 2.289422219600623, 0.09856183064560797]
Median solution: [array([0.01505601, 0.00600448]), 0.08669694299078863, 0.007560510998008585]
Mean solution: 0.024979725716286053
Standard deviation: 0.03302367765028963


## $(\mu, \lambda)- EE*$ (*improved) (40 puntos)
\*by using intermediate local recombination

In [12]:
def intermediate_offspring(parents_sols):
    mean_x_s = (parents_sols[0][0]+parents_sols[1][0])/2
    mean_sigma = (parents_sols[0][1]+parents_sols[1][1])/2
    return [mean_x_s,mean_sigma,0] #f(x) calculated in the next step

In [245]:
n = 3
rand_sol = random_solution(n)
mutated_solution = mutate(rand_sol, uniform(), n)
print(rand_sol)
print(mutated_solution)
print(intermediate_offspring([rand_sol, mutated_solution]))

[array([-16.46768991,  24.12962874,  13.55950631]), 0.4691560113202292, 22.053332267927736]
[array([-16.50069104,  24.12516481,  13.57374353]), 0.024368178054107275, 22.04531534051485]
[array([-16.48419047,  24.12739677,  13.56662492]), 0.24676209468716825, 0]


In [37]:
def EEmu_coma_lambda_2(n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None):
    population = [random_solution(n) for i in range(mu)]
    for gen in range(gens):
        offspring = []
        for kid in range(lambda_):
            parents = rand_select_parents(mu, population, 2) # here can I select the same parent twice? I guess because there are more children than parents.
            offspring_i = intermediate_offspring(parents)
            offspring_i = mutate(offspring_i,sigma,n,tau)
            offspring_i[0] = check_domain(offspring_i[0],-30,30)
            offspring.append(offspring_i)
        population = selection(lambda_, offspring)
    return population[0]

In [38]:
EEmu_coma_lambda_2(2, 200, 0.0001, 100, 700)

[array([ 0.00997031, -0.01005846]), 0.19758153496021075, 0.005775286481995767]

## Statistical analysis (20 puntos)

In [44]:
def m_EEmu_coma_lambda_2(m, n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None):
    m_solutions = []
    for i in range(m):
        EE_sol = EEmu_coma_lambda_2(n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None)
        m_solutions.append(EE_sol)
    selection(m, m_solutions)
    return m_solutions

In the function above I've already ordered the solutions from best to worst.

In [45]:
m_EEmu_coma_lambda_2(5, 2, 200, 0.0001, 100, 700)

[[array([-0.00347152, -0.00072118]),
  0.03864779637220995,
  0.000362383946932443],
 [array([0.00552201, 0.01066766]), 0.041723388016612316, 0.004155711644444171],
 [array([-0.01625152, -0.00026338]), 0.2721498296229934, 0.007600538936561652],
 [array([ 0.0235565 , -0.00049584]), 0.22141248940546904, 0.01593643041442805],
 [array([-0.02388924, -0.00260922]), 0.252055630133475, 0.016575827378737973]]

In [124]:
_20sols = m_EEmu_coma_lambda_2(20, 2, 200, 0.0001, 100, 700)

In [125]:
median_sol(20,_20sols)

[array([0.00761274, 0.01444829]), 0.2667057544293175, 0.007675178488153467]

In [126]:
mean_sol_f(_20sols)

0.015993450944815014

In [127]:
std_sol_f(_20sols)

0.01805897545811264

In [131]:
def m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None):
    m_sols =  m_EEmu_coma_lambda_2(m, n, gens, epsilon, mu, lambda_,sigma=uniform(), tau=None)
    print('Best solution:',m_sols[0])
    print('Worst solution:', m_sols[-1])
    print('Median solution:', median_sol(m,m_sols))
    print('Mean solution:', mean_sol_f(m_sols))
    print('Standard deviation:', std_sol_f(m_sols))

In [132]:
m = 20
m_EEmu_coma_lambda_2_statistical_analysis(m, 2, 200, 0.0001, 100, 700)

Best solution: [array([0.00191279, 0.00078039]), 0.03803612850255615, 0.00012302980640699346]
Worst solution: [array([0.05817009, 0.0094691 ]), 0.3334603303149343, 0.09757967118699584]
Median solution: [array([-0.01927921, -0.0094194 ]), 0.6561170336932433, 0.013232092327588152]
Mean solution: 0.017632874407698028
Standard deviation: 0.023446635588721457


In [133]:
m = 20
m_EEmu_coma_lambda_2_statistical_analysis(m, 2, 20, 0.0001, 100, 700)

Best solution: [array([ 8.45435324e-05, -1.21199702e-04]), 0.0031239826004583744, 6.29524273154658e-07]
Worst solution: [array([ 0.0376557 , -0.06415227]), 0.7299136181570685, 0.15396071971214242]
Median solution: [array([0.01977025, 0.00022151]), 0.2393777949500635, 0.011235704088985887]
Mean solution: 0.02182587666945741
Standard deviation: 0.034430756409653304


### Examples varying only number of decision variables 

#### Example with 5 decision variables

In [134]:
m = 10
m_EEmu_coma_lambda_2_statistical_analysis(m, 5, 20, 0.0001, 100, 700)

Best solution: [array([ 0.02648057, -0.00040961,  0.05718651, -0.08346873,  0.01254526]), 0.1333335622939757, 0.12337722446640642]
Worst solution: [array([-0.23038721,  0.03245943,  0.13899735,  0.12553355, -0.19384062]), 0.3777631894232656, 1.070609345469205]
Median solution: [array([-0.07106573, -0.02854435,  0.05650822,  0.00666478, -0.20934327]), 0.5729321178847202, 0.5012441841571946]
Mean solution: 0.556909550406941
Standard deviation: 0.32125967620966345


#### Example with 10 decision variables

In [135]:
m = 10
m_EEmu_coma_lambda_2_statistical_analysis(m, 10, 20, 0.0001, 100, 700)

Best solution: [array([ 0.05557385,  0.04560238, -0.06888201, -0.00679602,  0.02429066,
        0.10001206,  0.11594128,  0.10536936,  0.07106878,  0.0071611 ]), 0.07483070310187373, 0.2691445017285372]
Worst solution: [array([-0.04652473, -0.15032351, -0.62332628,  0.14967017, -0.36694653,
       -0.09037512, -0.63104782, -0.16252678, -0.27570804, -0.18957077]), 0.6965884515080196, 1.980733115688928]
Median solution: [array([ 0.16871935, -0.21712873,  0.05502283, -0.00914295, -0.38072105,
       -0.02827614, -0.30082158,  0.23418936,  0.80233784,  0.00282029]), 0.5138900944965723, 1.6288010005667322]
Mean solution: 1.175667400265441
Standard deviation: 0.6631672380900387


#### Example with 20 dicision variables

In [136]:
m = 10
m_EEmu_coma_lambda_2_statistical_analysis(m, 20, 20, 0.0001, 100, 700)

Best solution: [array([ 0.24318616, -0.03782471, -0.19188319, -0.01165246, -0.07525335,
       -0.07716254, -0.03698694, -0.01407658,  0.08770343, -0.07598361,
        0.0254767 ,  0.0301144 ,  0.10345013, -0.0201304 , -0.0888362 ,
       -0.16911753, -0.10690462, -0.13195186,  0.01317293, -0.27252504]), 0.1742552158344251, 0.6288652779890787]
Worst solution: [array([ 0.86725567, -0.85328343, -0.3434064 ,  0.01593352, -0.2292368 ,
       -0.01955856,  0.86603484, -0.79652277,  0.04508148, -0.78379223,
       -0.8893131 , -0.52620678, -0.67354775, -0.32140641, -0.03079048,
        0.07389731, -0.31992175, -0.87413341, -0.05585552, -0.13778712]), 0.7080960377876641, 2.4537000366761856]
Median solution: [array([-0.40298005, -0.47372179,  0.11234977,  0.00680029, -0.5072501 ,
       -0.76425585, -0.41465858,  0.15648327,  0.07594444,  1.06421026,
       -0.44438267,  0.09096147, -0.06048978,  0.8859793 ,  0.17782405,
       -0.0086719 , -0.10809384,  0.28778171, -0.31545452, -0.11665323]),

## Comparison:  (1+1)-EE      vs  $(\mu, \lambda)- EE$  vs  $(\mu, \lambda)- EE*$ (20 puntos)

In [144]:
m = 20
gens = 200
epsilon = 0.0001
k = 50
c = 0.8
mu = 100
lambda_ = 700

### 2 variables

In [145]:
n = 2

In [146]:
m_EE1plus1_statistical_analysis(m, n, uniform(), c, gens, k)

Best solution: [array([0.00303567, 0.05235813]), 0.544533748943318, 0.07765829939156932]
Worst solution: [array([ 14.90464519, -15.01019965]), 0.967460112137178, 20.229268667317346]
Median solution: [array([24.02574757,  6.99648806]), 0.23568874899702374, 20.018017672577127]
Mean solution: 19.034439649755672
Standard deviation: 4.3494618618793215


In [147]:
m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([-0.00040051,  0.00045345]), 0.032454402437197355, 1.055197523003315e-05]
Worst solution: [array([ 0.02804514, -0.03530157]), 0.5004941736418926, 0.05786889195939482]
Median solution: [array([-0.00814502,  0.00499551]), 0.21431284052477637, 0.002630383177593121]
Mean solution: 0.011658835961369452
Standard deviation: 0.01567999804979634


In [148]:
m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([9.82370854e-05, 8.71996403e-05]), 0.0028160529085921554, 4.974132932211717e-07]
Worst solution: [array([0.03646249, 0.02968456]), 0.19054583476748282, 0.06286829266950322]
Median solution: [array([ 0.00584692, -0.01452266]), 0.11548450685064772, 0.007053693682266715]
Mean solution: 0.013490825818326169
Standard deviation: 0.01591612788512225


### 5 variables

In [149]:
n = 5

In [150]:
m_EE1plus1_statistical_analysis(m, n, uniform(), c, gens, k)

Best solution: [array([-2.99745347, 10.00487062, -3.00007989, -5.99925008, 18.99143834]), 0.020351270434379898, 20.001116439369454]
Worst solution: [array([-27.09676979, -15.90822724,  19.73791967,  28.96597932,
        29.97901768]), 0.004849324016267587, 20.68355035988564]
Median solution: [array([-21.93675191,  19.98906853, -10.96059181, -29.86375213,
        13.94639113]), 0.6387961943680827, 20.264363738505207]
Mean solution: 20.256861285644632
Standard deviation: 0.19372212418078544


In [151]:
m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([ 0.02139912, -0.01642453, -0.0195737 , -0.01518669, -0.01303799]), 0.048346936475767044, 0.017362693254768136]
Worst solution: [array([-19.95236471, -20.97746761, -23.03607252,   1.1348169 ,
         1.9343832 ]), 0.19105025483024962, 20.2594925879909]
Median solution: [array([ 0.03164107,  0.1142565 , -0.15990544, -0.21883647,  0.06793592]), 0.2276593367684898, 0.827953363947302]
Mean solution: 3.5150506927459264
Standard deviation: 6.984980404933553


In [152]:
m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([-0.0103353 ,  0.00604072, -0.0135031 ,  0.01761681, -0.00394101]), 0.13411377601803515, 0.007498964811333497]
Worst solution: [array([-0.10119014,  0.09402465,  1.01753434,  0.01920602, -0.10208364]), 0.2195297734410397, 1.130549818988261]
Median solution: [array([-0.15953282, -0.0592702 , -0.10484707, -0.03449448,  0.06214142]), 0.21211997809730326, 0.4554976582314514]
Mean solution: 0.42155259215932145
Standard deviation: 0.27343652909742494


### 7 variables

In [153]:
n = 7

In [154]:
m_EE1plus1_statistical_analysis(m, n, uniform(), c, gens, k)

Best solution: [array([ -3.96608403, -24.00847412,  26.97484991,  12.01662325,
       -24.97687816,  28.09301792,  12.03087429]), 0.08869032225724827, 20.09057584553388]
Worst solution: [array([-15.08362708,   2.28624354, -22.08178176, -12.14928185,
       -28.1469066 ,  26.11432842,  -4.03700069]), 0.9476864673189376, 20.83502519284477]
Median solution: [array([ 15.87610469, -14.86989007, -22.97900859,  11.12700467,
        12.79684498,  -3.96586389,  -4.99223553]), 0.925634966081488, 20.570361332601287]
Mean solution: 20.510215514139635
Standard deviation: 0.21060379245179744


In [155]:
m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([-0.02646321, -0.12382004,  0.13886638,  1.05850504,  0.15823656,
        0.08213989, -0.26756948]), 0.5567259687140075, 1.501403553481016]
Worst solution: [array([-26.94258367,   4.89880877, -30.        ,   6.87460735,
       -29.28711163, -17.90686996,  -7.01427976]), 0.9299532975266732, 20.797655117653356]
Median solution: [array([-19.03217195, -23.96267043,  14.84361478,  14.09029251,
        26.98732456,  -2.05211179, -13.13216868]), 0.24696532226269027, 20.372370728561034]
Mean solution: 18.5136806346865
Standard deviation: 5.6732069877872


In [156]:
m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([ 0.00105795, -0.00468953, -0.00068375,  0.02095468,  0.00341292,
        0.01228478,  0.01128683]), 0.0441563052195718, 0.0061872285430131235]
Worst solution: [array([-0.24702915, -0.25987152, -0.85833517,  0.12451195, -0.11205252,
       -0.03763985,  0.2321027 ]), 1.3263408106524852, 1.6865870148214026]
Median solution: [array([ 0.1278985 , -0.07976832, -0.2001537 , -0.12674895,  0.12637001,
       -0.04151412,  0.10957941]), 0.394489485208407, 0.7322635731497589]
Mean solution: 0.755422049406268
Standard deviation: 0.4718625808026292


### 10 variables

In [157]:
n = 10

In [158]:
m_EE1plus1_statistical_analysis(m, n, uniform(), c, gens, k)

Best solution: [array([ 19.0057207 ,  22.99407849, -10.01081567, -28.04670925,
        23.04819239,  -1.94220341,  -1.05062527,  27.93897388,
        -5.94603781,  15.9451685 ]), 0.059229121652967565, 20.10543287657524]
Worst solution: [array([-1.50510973e-02, -1.29612022e+00,  1.92151940e+01,  4.12704483e+00,
       -2.10150982e+01,  2.03594724e+00,  2.99752139e+01,  2.28321880e+01,
        1.91161836e-01, -4.72338026e+00]), 0.47629981035811253, 21.024216383071227]
Median solution: [array([-22.09018929, -15.92980492,  -0.8933553 , -26.99019058,
        18.26578841,  27.11614601, -27.81861865,  16.92122098,
        -6.75065739, -18.11241626]), 0.34955838075852963, 20.85210275997425]
Mean solution: 20.718777033788722
Standard deviation: 0.29779632750478363


In [159]:
m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([-13.01576011, -18.97601824,  26.01730369, -14.01016624,
        26.97609673,   3.00932138,   2.01925995,  -2.99142144,
       -20.00029462,   4.97763236]), 0.034227594370630846, 20.015118265280922]
Worst solution: [array([ 14.15230426,  18.86047065,  17.04673044,   1.72070025,
        18.98854347, -24.91737738,   5.96633507, -20.31463923,
         6.93128091,  27.84794051]), 0.39989313806615606, 20.910021135626792]
Median solution: [array([ -4.23859616, -21.12950695, -24.00560666,  15.71617608,
       -18.03106236,  -2.03708117, -14.95966676,  -0.08292645,
         7.16335016,  -9.93060239]), 0.7060408777025564, 20.752171303003948]
Mean solution: 20.6451851508918
Standard deviation: 0.2750646895765319


In [160]:
m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([ 0.12360191,  0.02175573, -0.061265  ,  0.04029539,  0.01703255,
       -0.06964967, -0.10243275,  0.05974895, -0.04197548, -0.06329747]), 0.11705392176562004, 0.24826053269876214]
Worst solution: [array([-0.00093066, -0.00604415,  0.22521946,  0.8151502 ,  0.90481739,
       -0.07124083,  0.0954325 ,  0.07340004,  0.49135049, -0.56504053]), 1.0059563375186222, 2.041745130945014]
Median solution: [array([-0.10744567,  0.27024217,  0.00287608, -0.03652062,  0.59166647,
       -0.08957054, -0.21480128,  0.00432532,  0.28498419, -0.06460638]), 0.5340136823813879, 1.3703485868477405]
Mean solution: 1.0948226844021565
Standard deviation: 0.5084865606979673


m = 20     
### best solution
| Algorithm/Variables | $$2$$ |  $$5$$ | $$7$$ | $$10$$ |
| --- | --- | --- | --- | --- |
| $(1+1)-EE$ | 0.07765829939156932 | 20.001116439369454 | 20.09057584553388 | 20.10543287657524 |
| $(\mu, \lambda)- EE$ | 1.055197523003315e-05 | 0.017362693254768136 | 1.501403553481016 | 20.015118265280922 |
| $(\mu, \lambda)- EE*$ | 4.974132932211717e-07 | 0.007498964811333497 | 0.0061872285430131235 | 0.24826053269876214 |

### best solution
| Variables | (1+1)-EE |  $$(\mu, \lambda)- EE$$ | $$(\mu, \lambda)- EE*$$ |
| --- | --- | --- | --- |
| 2 |  |  |  |
| 5 |  |  |  |
| 7 |  |  |  |
| 10 |  |  |  |    

### worst solution
| Algorithms/Variables | 2 |  5 | 7 | 10 |
| --- | --- | --- | --- | --- |
| $(1+1)-EE$ |  |  |  |  |
| $(\mu, \lambda)- EE$ |  |  |  |  |
| $(\mu, \lambda)- EE*$ |  |  |  |  |   

### mean solution
| Algorithms/Variables | 2 |  5 | 7 | 10 |
| --- | --- | --- | --- | --- |
| $(1+1)-EE$ |  |  |  |  |
| $(\mu, \lambda)- EE$ |  |  |  |  |
| $(\mu, \lambda)- EE*$ |  |  |  |  |    

### std solution  
| Algorithms/Variables | 2 |  5 | 7 | 10 |
| --- | --- | --- | --- | --- |
| $(1+1)-EE$ |  |  |  |  |
| $(\mu, \lambda)- EE$ |  |  |  |  |
| $(\mu, \lambda)- EE*$ |  |  |  |  |